<a href="https://colab.research.google.com/github/IMOKURI/Hungry-Geese/blob/main/hungry_geese_train_by_episode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About this notebook ...

## Prepare for Colab

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules


In [36]:
COMPETE = "hungry-geese"
DATASETS = [
    "imokuri/hungrygeeseepisode",
]
KERNEL_OUTPUTS = [
    # "imokuri/notebook8d5846e909",
    # "imokuri/notebook42cdc46ffc",
]
PACKAGES = []


In [3]:
if IN_COLAB:
    # Work around for python2 exception.
    !python2 -m pip uninstall kaggle -y
    !python3 -m pip uninstall kaggle -y
    !python3 -m pip install -U -q kaggle


Uninstalling kaggle-1.5.12:
  Successfully uninstalled kaggle-1.5.12


In [37]:
if IN_COLAB:
    !pip install -q -U git+https://github.com/IMOKURI/kaggle_on_google_colab.git

    from kaggle_on_google_colab import setup
    kaggle = setup.Setup()
    kaggle.dirs(COMPETE)

    # !kaggle competitions download -p /content/zip {COMPETE}
    # !unzip -q -n /content/zip/{COMPETE}.zip -d /content/{COMPETE}/input/{COMPETE}

    for dataset in DATASETS:
        dataset_name = dataset.split("/")[-1]
        !kaggle datasets download -p /content/zip {dataset}
        !unzip -q -n /content/zip/{dataset_name}.zip -d /content/{COMPETE}/input/{dataset_name}

    for kernel in KERNEL_OUTPUTS:
        kernel_name = kernel.split("/")[-1]
        !kaggle kernels output -p /content/{COMPETE}/input/{kernel_name} {kernel}

    for package in PACKAGES:
        !pip install -q {package}

    %cd /content/{COMPETE}/output


Download 100%.
Output file downloaded to /content/hungry-geese/input/notebook8d5846e909/19425049.json
Output file downloaded to /content/hungry-geese/input/notebook8d5846e909/20467787.json
Output file downloaded to /content/hungry-geese/input/notebook8d5846e909/20467787_info.json
Output file downloaded to /content/hungry-geese/input/notebook8d5846e909/20468380.json
Output file downloaded to /content/hungry-geese/input/notebook8d5846e909/20468380_info.json
Output file downloaded to /content/hungry-geese/input/notebook8d5846e909/20468874.json
Output file downloaded to /content/hungry-geese/input/notebook8d5846e909/20468874_info.json
Output file downloaded to /content/hungry-geese/input/notebook8d5846e909/20468933.json
Output file downloaded to /content/hungry-geese/input/notebook8d5846e909/20468933_info.json
Output file downloaded to /content/hungry-geese/input/notebook8d5846e909/20468946.json
Output file downloaded to /content/hungry-geese/input/notebook8d5846e909/20468946_info.json
Out

## Library

In [5]:
import glob
import json
import math
import os
import random
import time
import warnings
from contextlib import contextmanager

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts, ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm

In [6]:
warnings.filterwarnings("ignore")

## Load Data

In [7]:
BASE_DIR = "../input/hungrygeeseepisode/hungry-geese-episode/"
OUTPUT_DIR = "pre-models/"

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [8]:
paths = [path for path in glob.glob(BASE_DIR + "*.json") if "info" not in path]
print(len(paths))

1001


## Config

In [9]:
class Config:
    seed = 440

    n_class = 4
    n_fold = 5

    gradient_accumulation_steps = 1
    max_grad_norm = 1000

    num_workers = 4
    batch_size = 800

    scheduler = "CosineAnnealingWarmRestarts"
    # factor = 0.2  # ReduceLROnPlateau
    # patience = 4  # ReduceLROnPlateau
    # eps = 1e-6  # ReduceLROnPlateau
    # T_max = 10  # CosineAnnealingLR
    T_0 = 10  # CosineAnnealingWarmRestarts

    criterion = "CrossEntropyLoss"
    lr = 1e-3
    min_lr = 5e-5
    weight_decay = 0

    epochs = 10
    model_name = "geese_net"

    print_freq = 100

    train = True
    debug = False
    apex = False

In [10]:
if Config.debug:
    Config.epochs = 1

In [11]:
if Config.apex:
    from apex import amp

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Utils

In [13]:
@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f"[{name}] start")
    yield
    LOGGER.info(f"[{name}] done in {time.time() - t0:.0f} s.")


def init_logger(log_file=OUTPUT_DIR + "train.log"):
    from logging import INFO, FileHandler, Formatter, StreamHandler, getLogger

    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger


LOGGER = init_logger()


def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_torch(seed=Config.seed)

In [14]:
def reverse_ns(y):
    if y == 0:
        return 1
    if y == 1:
        return 0
    return y

def reverse_we(y):
    if y == 2:
        return 3
    if y == 3:
        return 2
    return y

def reverse_nswe(y):
    return reverse_ns(reverse_we(y))

## Observation

In [15]:
def make_input(obses):
    b = np.zeros((17, 7 * 11), dtype=np.float32)
    obs = obses[-1]

    for p, pos_list in enumerate(obs["geese"]):
        pid = (p - obs["index"]) % 4

        # head position
        for pos in pos_list[:1]:
            b[0 + pid, pos] = 1
        # tip position
        for pos in pos_list[-1:]:
            b[4 + pid, pos] = 1
        # whole position
        for pos in pos_list:
            b[8 + pid, pos] = 1

    # previous head position
    if len(obses) > 1:
        obs_prev = obses[-2]
        for p, pos_list in enumerate(obs_prev["geese"]):
            for pos in pos_list[:1]:
                b[12 + (p - obs["index"]) % 4, pos] = 1

    # food
    for pos in obs["food"]:
        b[16, pos] = 1

    return b.reshape(-1, 7, 11)

In [16]:
def observation_num_step(obses):
    b = np.zeros((7, 11), dtype=np.float32)
    obs = obses[-1]

    num_step = obs["step"]  # 0-198
    b[0, 0] = num_step / 198

    return b.reshape(1, 7, 11)

## Data

In [17]:
def create_dataset_from_json(filepath, json_object=None, standing=0):
    if json_object is None:
        json_open = open(path, "r")
        json_load = json.load(json_open)
    else:
        json_load = json_object

    try:
        winner_index = np.argmax(np.argsort(json_load["rewards"]) == 3 - standing)

        obses = []
        X = []
        y = []
        actions = {"NORTH": 0, "SOUTH": 1, "WEST": 2, "EAST": 3}

        for i in range(len(json_load["steps"]) - 1):
            if json_load["steps"][i][winner_index]["status"] == "ACTIVE":
                y_ = json_load["steps"][i + 1][winner_index]["action"]
                if y_ is not None:
                    step = json_load["steps"][i]
                    step[winner_index]["observation"]["geese"] = step[0]["observation"]["geese"]
                    step[winner_index]["observation"]["food"] = step[0]["observation"]["food"]
                    step[winner_index]["observation"]["step"] = step[0]["observation"]["step"]
                    obses.append(step[winner_index]["observation"])
                    y.append(actions[y_])

                    y.append(reverse_ns(actions[y_]))  # 上下反転
                    y.append(reverse_we(actions[y_]))  # 左右反転
                    y.append(reverse_nswe(actions[y_]))  # 上下左右反転

        for j in range(len(obses)):
            # X_ = make_input(obses[: j + 1])

            X_ = []
            X_.append(make_input(obses[: j + 1]))
            # X_.append(observation_num_step(obses[: j + 1]))
            X_ = np.concatenate(X_)

            X.append(X_)

            X.append(X_[:, ::-1, :])  # 上下反転
            X.append(X_[:, :, ::-1])  # 左右反転
            X.append(X_[:, ::-1, ::-1])  # 上下左右反転

        X = np.array(X, dtype=np.float32)  # [starting_step:]
        y = np.array(y, dtype=np.uint8)  # [starting_step:]

        return X, y
    except:
        return 0, 0

In [18]:
X_train = []
y_train = []

for path in tqdm(paths[: int(len(paths))]):
    X, y = create_dataset_from_json(path, standing=0)  # use only winners' moves
    if X is not 0:
        X_train.append(X)
        y_train.append(y)
        
X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)

print(f"Num episode: {len(X_train)}")


Num episode: 642304


In [19]:
# TODO: データをuniqueにしたいがメモリエラーになってしまう。

# X_train, unique_index = np.unique(X_train, axis=0, return_index=True)  # remove duplicate
# y_train = y_train[unique_index]

# y_train = np.eye(4, dtype="uint8")[y_train]  # to categorical

# print(f"Num episode: {len(X_train)}")

In [20]:
if Config.debug:
    X_train = X_train[:1000]
    y_train = y_train[:1000]

In [21]:
y_df = pd.DataFrame(y_train)
y_df.columns = ["action"]
y_df

,action
0,1
1,0
2,1
3,0
4,1
...,...
642299,0
642300,1
642301,0
642302,1


## CV Split

In [22]:
folds = y_df.copy()
Fold = StratifiedKFold(n_splits=Config.n_fold, shuffle=True, random_state=Config.seed)
for n, (train_index, val_index) in enumerate(Fold.split(folds, folds["action"])):
    folds.loc[val_index, "fold"] = int(n)
folds["fold"] = folds["fold"].astype(int)
print(folds.groupby(["fold", "action"]).size())

fold  action
0     0         30728
      1         30729
      2         33502
      3         33502
1     0         30728
      1         30729
      2         33502
      3         33502
2     0         30729
      1         30728
      2         33502
      3         33502
3     0         30729
      1         30728
      2         33502
      3         33502
4     0         30728
      1         30728
      2         33502
      3         33502
dtype: int64


## Dataset

In [23]:
class TrainDataset(Dataset):
    def __init__(self, array, label):
        self.array = array
        self.label = label

    def __len__(self):
        return self.array.shape[0]

    def __getitem__(self, idx):
        return self.array[idx], torch.tensor(self.label[idx]).long()


class TestDataset(Dataset):
    def __init__(self, array):
        self.array = array

    def __len__(self):
        return self.array.shape[0]

    def __getitem__(self, idx):
        return self.array[idx]

In [24]:
# Test

train_ds = TrainDataset(X_train, y_train)

for i in range(1):
    obs, action = train_ds[i]
    print(obs.shape, action)

(17, 7, 11) tensor(1)


## Model

In [25]:
class TorusConv2d(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, bn):
        super().__init__()
        self.edge_size = (kernel_size[0] // 2, kernel_size[1] // 2)
        self.conv = nn.Conv2d(input_dim, output_dim, kernel_size=kernel_size)
        self.bn = nn.BatchNorm2d(output_dim) if bn else None

    def forward(self, x):
        h = torch.cat([x[:, :, :, -self.edge_size[1] :], x, x[:, :, :, : self.edge_size[1]]], dim=3)
        h = torch.cat([h[:, :, -self.edge_size[0] :], h, h[:, :, : self.edge_size[0]]], dim=2)
        h = self.conv(h)
        h = self.bn(h) if self.bn is not None else h
        return h

In [26]:
class GeeseNet(nn.Module):
    def __init__(self):
        super().__init__()
        layers, filters = 12, 32
        self.conv0 = TorusConv2d(17, filters, (3, 3), True)
        self.blocks = nn.ModuleList([TorusConv2d(filters, filters, (3, 3), True) for _ in range(layers)])

        self.conv_p = TorusConv2d(filters, filters, (3, 3), True)
        self.conv_v = TorusConv2d(filters, filters, (3, 3), True)

        self.head_p = nn.Linear(filters, 4, bias=False)
        self.head_v1 = nn.Linear(filters * 2, filters, bias=False)
        self.head_v2 = nn.Linear(filters, 1, bias=False)

    def forward(self, x, _=None):
        h = F.relu_(self.conv0(x))
        for block in self.blocks:
            h = F.relu_(h + block(h))

        h_p = F.relu_(self.conv_p(h))
        h_head_p = (h_p * x[:, :1]).view(h_p.size(0), h_p.size(1), -1).sum(-1)
        p = self.head_p(h_head_p)

        h_v = F.relu_(self.conv_v(h))
        h_head_v = (h_v * x[:, :1]).view(h_v.size(0), h_v.size(1), -1).sum(-1)
        h_avg_v = h_v.view(h_v.size(0), h_v.size(1), -1).mean(-1)

        h_v = F.relu_(self.head_v1(torch.cat([h_head_v, h_avg_v], 1)))
        v = torch.tanh(self.head_v2(h_v))

        return {"policy": p, "value": v}

In [27]:
class GeeseNetAlpha(nn.Module):
    def __init__(self):
        super().__init__()
        layers, filters = 12, 64
        self.conv0 = TorusConv2d(17, filters, (3, 3), True)
        self.blocks = nn.ModuleList([TorusConv2d(filters, filters, (3, 3), True) for _ in range(layers)])

        self.conv_p = TorusConv2d(filters, filters, (3, 3), True)
        # self.conv_v = TorusConv2d(filters, filters, (3, 3), True)

        self.head_p1 = nn.Linear(filters * 2 + 77, filters, bias=False)
        self.head_p2 = nn.Linear(filters, 4, bias=False)
        # self.head_v1 = nn.Linear(filters * 2, filters, bias=False)
        # self.head_v2 = nn.Linear(filters, 1, bias=False)

    def forward(self, x, _=None):
        # x = x[:, :-1]
        # num_step = x[:, -1, 0, 0].view(x.size(0), 1)

        h = F.relu_(self.conv0(x))
        for block in self.blocks:
            h = F.relu_(h + block(h))

        h_p = F.relu_(self.conv_p(h))
        h_head_p = (h_p * x[:, :1]).view(h_p.size(0), h_p.size(1), -1).sum(-1)
        h_avg_p1 = h_p.view(h_p.size(0), h_p.size(1), -1).mean(-1)
        h_avg_p2 = h_p.view(h_p.size(0), h_p.size(1), -1).mean(1)

        h_p = F.relu_(self.head_p1(torch.cat([h_head_p, h_avg_p1, h_avg_p2], 1)))
        p = self.head_p2(h_p)

        # h_v = F.relu_(self.conv_v(h))
        # h_head_v = (h_v * x[:, :1]).view(h_v.size(0), h_v.size(1), -1).sum(-1)
        # h_avg_v = h_v.view(h_v.size(0), h_v.size(1), -1).mean(-1)

        # h_v = F.relu_(self.head_v1(torch.cat([h_head_v, h_avg_v], 1)))
        # v = torch.tanh(self.head_v2(h_v))

        return {"policy": p}  # "value": v

In [28]:
# Test

model = GeeseNetAlpha()
# print(model)

params = sum(p.numel() for p in model.parameters())
print(f"params: {params:,}")

train_ds = TrainDataset(X_train, y_train)
train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)

for obs, action in train_loader:
    output = model(obs)
    print(output)
    print(f"{torch.argmax(output['policy'], dim=1)}")
    break

params: 505,088
{'policy': tensor([[-0.0482,  0.0622,  0.0121,  0.0654],
        [ 0.0047, -0.1130, -0.0873,  0.1382],
        [-0.0846,  0.0202,  0.1040, -0.0044],
        [-0.0422, -0.0436,  0.0040,  0.0635]], grad_fn=<MmBackward>)}
tensor([3, 3, 2, 3])


## Loss

## Scoring

In [29]:
def get_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

## Helper functions

In [30]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))

In [31]:
def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()

    # switch to train mode
    model.train()
    start = end = time.time()
    global_step = 0

    for step, (obs, action) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        obs = obs.to(device)
        action = action.to(device)
        batch_size = action.size(0)

        y_preds = model(obs.float())["policy"]

        loss = criterion(y_preds, action)

        # record loss
        losses.update(loss.item(), batch_size)
        if Config.gradient_accumulation_steps > 1:
            loss = loss / Config.gradient_accumulation_steps
        if Config.apex:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
        else:
            loss.backward()

        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), Config.max_grad_norm)

        if (step + 1) % Config.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if step % Config.print_freq == 0 or step == (len(train_loader) - 1):
            print(
                f"Epoch: [{epoch + 1}][{step}/{len(train_loader)}] "
                f"Elapsed {timeSince(start, float(step + 1) / len(train_loader)):s} "
                f"Loss: {losses.val:.4f}({losses.avg:.4f}) "
                f"Grad: {grad_norm:.4f} "
                f"LR: {scheduler.get_last_lr()[0]:.6f}  "
            )

    return losses.avg

In [32]:
def valid_fn(valid_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()

    # switch to evaluation mode
    model.eval()
    preds = []
    start = end = time.time()

    for step, (obs, action) in enumerate(valid_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        obs = obs.to(device)
        action = action.to(device)
        batch_size = action.size(0)

        # compute loss
        with torch.no_grad():
            y_preds = model(obs)["policy"]

        loss = criterion(y_preds, action)
        losses.update(loss.item(), batch_size)

        # record accuracy
        preds.append(y_preds.softmax(1).to("cpu").numpy())
        if Config.gradient_accumulation_steps > 1:
            loss = loss / Config.gradient_accumulation_steps

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if step % Config.print_freq == 0 or step == (len(valid_loader) - 1):
            print(
                f"EVAL: [{step}/{len(valid_loader)}] "
                f"Elapsed {timeSince(start, float(step + 1) / len(valid_loader)):s} "
                f"Loss: {losses.val:.4f}({losses.avg:.4f}) "
            )
    predictions = np.concatenate(preds)
    return losses.avg, predictions

## Train loop

In [33]:
def train_loop(folds, fold):

    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # Data Loader
    # ====================================================
    X_train_folds = X_train[folds["fold"] != fold]
    X_valid_folds = X_train[folds["fold"] == fold]

    y_train_folds = y_train[folds["fold"] != fold]
    y_valid_folds = y_train[folds["fold"] == fold]

    y_df_train_folds = y_df[folds["fold"] != fold]
    y_df_valid_folds = y_df[folds["fold"] == fold]

    train_dataset = TrainDataset(X_train_folds, y_train_folds)
    valid_dataset = TrainDataset(X_valid_folds, y_valid_folds)

    train_loader = DataLoader(
        train_dataset,
        batch_size=Config.batch_size,
        shuffle=True,
        num_workers=Config.num_workers,
        pin_memory=True,
        drop_last=True,
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=Config.batch_size,
        shuffle=False,
        num_workers=Config.num_workers,
        pin_memory=True,
        drop_last=False,
    )

    # ====================================================
    # Scheduler
    # ====================================================
    def get_scheduler(optimizer):
        if Config.scheduler == "ReduceLROnPlateau":
            scheduler = ReduceLROnPlateau(
                optimizer, mode="min", factor=Config.factor, patience=Config.patience, verbose=True, eps=Config.eps
            )
        elif Config.scheduler == "CosineAnnealingLR":
            scheduler = CosineAnnealingLR(optimizer, T_max=Config.T_max, eta_min=Config.min_lr, last_epoch=-1)
        elif Config.scheduler == "CosineAnnealingWarmRestarts":
            scheduler = CosineAnnealingWarmRestarts(
                optimizer, T_0=Config.T_0, T_mult=1, eta_min=Config.min_lr, last_epoch=-1
            )
        return scheduler

    # ====================================================
    # model & optimizer
    # ====================================================
    model = GeeseNetAlpha()
    model.to(device)

    # Use multi GPU
    if device == torch.device("cuda") and not Config.apex:
        model = torch.nn.DataParallel(model)  # make parallel
        # torch.backends.cudnn.benchmark=True

    optimizer = Adam(model.parameters(), lr=Config.lr, weight_decay=Config.weight_decay, amsgrad=False)
    scheduler = get_scheduler(optimizer)

    # ====================================================
    # apex
    # ====================================================
    if Config.apex:
        model, optimizer = amp.initialize(model, optimizer, opt_level="O1", verbosity=0)

    # ====================================================
    # Criterion
    # ====================================================
    def get_criterion():
        if Config.criterion == "CrossEntropyLoss":
            criterion = nn.CrossEntropyLoss()
        return criterion

    criterion = get_criterion()

    # ====================================================
    # loop
    # ====================================================
    best_score = 0.0
    best_loss = np.inf

    for epoch in range(Config.epochs):

        start_time = time.time()

        # train
        avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)

        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(avg_val_loss)
        elif isinstance(scheduler, CosineAnnealingLR):
            scheduler.step()
        elif isinstance(scheduler, CosineAnnealingWarmRestarts):
            scheduler.step()

        # scoring
        score = get_score(y_valid_folds, preds.argmax(1))

        elapsed = time.time() - start_time

        LOGGER.info(
            f"Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s"
        )
        LOGGER.info(f"Epoch {epoch+1} - Accuracy: {score}")

        if score > best_score:
            best_score = score
            LOGGER.info(f"Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model")
            torch.save(
                {"model": model.state_dict(), "preds": preds}, OUTPUT_DIR + f"{Config.model_name}_fold{fold}_best.pth"
            )

        if epoch == Config.epochs - 1:
            LOGGER.info(f"Epoch {epoch+1} - Save final model")
            torch.save(
                {"model": model.state_dict(), "preds": preds}, OUTPUT_DIR + f"{Config.model_name}_fold{fold}_final.pth"
            )

    check_point = torch.load(OUTPUT_DIR + f"{Config.model_name}_fold{fold}_best.pth")

    y_df_valid_folds[[str(c) for c in range(Config.n_class)]] = check_point["preds"]
    y_df_valid_folds["preds"] = check_point["preds"].argmax(1)

    return y_df_valid_folds

## Main


In [34]:
def main():
    def get_result(result_df):
        preds = result_df["preds"].values
        labels = result_df["action"].values
        score = get_score(labels, preds)
        LOGGER.info(f"Score: {score:<.5f}")

    if Config.train:
        # train
        oof_df = pd.DataFrame()
        for fold in range(Config.n_fold):
            _oof_df = train_loop(folds, fold)
            oof_df = pd.concat([oof_df, _oof_df])
            LOGGER.info(f"========== fold: {fold} result ==========")
            get_result(_oof_df)
        # CV result
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        # save result
        oof_df.to_csv(OUTPUT_DIR + "oof_df.csv", index=False)

In [35]:
if __name__ == "__main__":
    main()

========== fold: 0 training ==========


Epoch: [1][0/642] Elapsed 0m 0s (remain 5m 37s) Loss: 1.3819(1.3819) Grad: 0.4928 LR: 0.001000  
Epoch: [1][100/642] Elapsed 0m 14s (remain 1m 19s) Loss: 0.6012(0.7205) Grad: 0.7432 LR: 0.001000  
Epoch: [1][200/642] Elapsed 0m 29s (remain 1m 3s) Loss: 0.5813(0.6668) Grad: 0.7146 LR: 0.001000  
Epoch: [1][300/642] Elapsed 0m 43s (remain 0m 49s) Loss: 0.5728(0.6383) Grad: 1.0060 LR: 0.001000  
Epoch: [1][400/642] Elapsed 0m 57s (remain 0m 34s) Loss: 0.5191(0.6216) Grad: 0.6914 LR: 0.001000  
Epoch: [1][500/642] Elapsed 1m 12s (remain 0m 20s) Loss: 0.5174(0.6073) Grad: 0.4872 LR: 0.001000  
Epoch: [1][600/642] Elapsed 1m 26s (remain 0m 5s) Loss: 0.5729(0.5964) Grad: 0.8798 LR: 0.001000  
Epoch: [1][641/642] Elapsed 1m 32s (remain 0m 0s) Loss: 0.5589(0.5926) Grad: 0.6233 LR: 0.001000  
EVAL: [0/161] Elapsed 0m 0s (remain 1m 32s) Loss: 0.4899(0.4899) 
EVAL: [100/161] Elapsed 0m 4s (remain 0m 2s) Loss: 0.5256(0.5318) 


Epoch 1 - avg_train_loss: 0.5926  avg_val_loss: 0.5312  time: 99s
Epoch 1 - Accuracy: 0.7617019951580636
Epoch 1 - Save Best Score: 0.7617 Model


EVAL: [160/161] Elapsed 0m 6s (remain 0m 0s) Loss: 0.5655(0.5312) 
Epoch: [2][0/642] Elapsed 0m 0s (remain 7m 6s) Loss: 0.5176(0.5176) Grad: 0.7985 LR: 0.000976  
Epoch: [2][100/642] Elapsed 0m 14s (remain 1m 20s) Loss: 0.5187(0.5179) Grad: 0.8288 LR: 0.000976  
Epoch: [2][200/642] Elapsed 0m 29s (remain 1m 4s) Loss: 0.5030(0.5193) Grad: 0.5762 LR: 0.000976  
Epoch: [2][300/642] Elapsed 0m 43s (remain 0m 49s) Loss: 0.5133(0.5180) Grad: 0.4613 LR: 0.000976  
Epoch: [2][400/642] Elapsed 0m 57s (remain 0m 34s) Loss: 0.4793(0.5176) Grad: 0.4534 LR: 0.000976  
Epoch: [2][500/642] Elapsed 1m 12s (remain 0m 20s) Loss: 0.4810(0.5165) Grad: 0.3822 LR: 0.000976  
Epoch: [2][600/642] Elapsed 1m 26s (remain 0m 5s) Loss: 0.5026(0.5161) Grad: 0.9104 LR: 0.000976  
Epoch: [2][641/642] Elapsed 1m 32s (remain 0m 0s) Loss: 0.5636(0.5160) Grad: 0.9159 LR: 0.000976  
EVAL: [0/161] Elapsed 0m 0s (remain 1m 27s) Loss: 0.4650(0.4650) 
EVAL: [100/161] Elapsed 0m 4s (remain 0m 2s) Loss: 0.5111(0.5103) 


Epoch 2 - avg_train_loss: 0.5160  avg_val_loss: 0.5107  time: 99s
Epoch 2 - Accuracy: 0.7714092214757786
Epoch 2 - Save Best Score: 0.7714 Model


...

Epoch 8 - avg_train_loss: 0.4148  avg_val_loss: 0.4966  time: 98s
Epoch 8 - Accuracy: 0.7833972956772872


EVAL: [160/161] Elapsed 0m 6s (remain 0m 0s) Loss: 0.4935(0.4966) 
Epoch: [9][0/642] Elapsed 0m 0s (remain 4m 29s) Loss: 0.3909(0.3909) Grad: 0.5720 LR: 0.000105  
Epoch: [9][100/642] Elapsed 0m 14s (remain 1m 18s) Loss: 0.3927(0.3952) Grad: 0.6553 LR: 0.000105  
Epoch: [9][200/642] Elapsed 0m 28s (remain 1m 3s) Loss: 0.4109(0.3962) Grad: 0.7140 LR: 0.000105  
Epoch: [9][300/642] Elapsed 0m 43s (remain 0m 48s) Loss: 0.4048(0.3967) Grad: 0.6898 LR: 0.000105  
Epoch: [9][400/642] Elapsed 0m 57s (remain 0m 34s) Loss: 0.3811(0.3966) Grad: 0.6805 LR: 0.000105  
Epoch: [9][500/642] Elapsed 1m 11s (remain 0m 20s) Loss: 0.3738(0.3975) Grad: 0.6208 LR: 0.000105  
Epoch: [9][600/642] Elapsed 1m 25s (remain 0m 5s) Loss: 0.4046(0.3986) Grad: 0.6814 LR: 0.000105  
Epoch: [9][641/642] Elapsed 1m 31s (remain 0m 0s) Loss: 0.4482(0.3989) Grad: 0.7963 LR: 0.000105  
EVAL: [0/161] Elapsed 0m 0s (remain 0m 48s) Loss: 0.4445(0.4445) 
EVAL: [100/161] Elapsed 0m 4s (remain 0m 2s) Loss: 0.5128(0.5019) 


Epoch 9 - avg_train_loss: 0.3989  avg_val_loss: 0.5004  time: 98s
Epoch 9 - Accuracy: 0.783350588894684


EVAL: [160/161] Elapsed 0m 6s (remain 0m 0s) Loss: 0.4949(0.5004) 
Epoch: [10][0/642] Elapsed 0m 0s (remain 4m 28s) Loss: 0.4380(0.4380) Grad: 0.7612 LR: 0.000034  
Epoch: [10][100/642] Elapsed 0m 14s (remain 1m 18s) Loss: 0.3728(0.3872) Grad: 0.5851 LR: 0.000034  
Epoch: [10][200/642] Elapsed 0m 28s (remain 1m 3s) Loss: 0.4209(0.3870) Grad: 0.7678 LR: 0.000034  
Epoch: [10][300/642] Elapsed 0m 43s (remain 0m 48s) Loss: 0.3956(0.3877) Grad: 0.6339 LR: 0.000034  
Epoch: [10][400/642] Elapsed 0m 57s (remain 0m 34s) Loss: 0.3845(0.3874) Grad: 0.6586 LR: 0.000034  
Epoch: [10][500/642] Elapsed 1m 11s (remain 0m 20s) Loss: 0.3940(0.3876) Grad: 0.7304 LR: 0.000034  


KeyboardInterrupt: ignored